<a href="https://colab.research.google.com/github/blufzzz/Introspective-Neural-Networks/blob/master/Introspective_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
from torch import nn
import torchvision
from torchvision.models import resnet18

In [82]:
model = resnet18(pretrained = True)
X = torch.randn(1,3,256,256)
model.forward(X).shape

torch.Size([1, 1000])

In [0]:
class ICNN(nn.Module):
  
  def __init__(self, cnn_model, n_classes = False):
      super(ICNN, self).__init__()
      
      self.conv_features = [] # store conv_features for all passes
      self.model = cnn_model
      self.n_classes = n_classes
      self.sigmoid = nn.Sigmoid()

      self.model.fc = nn.Linear(in_features=512, out_features=1, bias=True)
      
  def forward(self, input, sigmoid = False):
      
      hook = lambda module, input, output: self.conv_features.append(output)
      self.model.avgpool.register_forward_hook(hook)
      output = self.model.forward(input)
      
#       # OVR classification
#       if n_classes:
#           # we took conv_features from the last forward pass
#           output = [fc(self.conv_features[-1]) for fc in self.fc_layers]
      
      if sigmoid:
          return  self.sigmoid(output)
      
      return output


In [86]:
X = torch.randn(2,3,256,256)
icnn_model = ICNN(model)
print(icnn_model.forward(X, sigmoid=True).shape)

torch.Size([2, 1])
